In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# Permission Warning 방지
!chmod 600 ~/.kaggle/kaggle.json

In [2]:
!kaggle competitions download -c playground-series-s4e8

 97% 80.0M/82.3M [00:04<00:00, 23.5MB/s]
100% 82.3M/82.3M [00:04<00:00, 17.5MB/s]


In [3]:
!unzip /content/playground-series-s4e8.zip

Archive:  /content/playground-series-s4e8.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [4]:
!pip install ray==2.10.0
!pip install autogluon.tabular
!pip install -U ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.1/312.1 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.8/234.8 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.1/139.1 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 111.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 6.5 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
ERROR: pip's dependency resolver does not currently take

In [5]:
import pandas as pd
from autogluon.tabular import TabularDataset, TabularPredictor

In [6]:
train_df = pd.read_csv("/content/train.csv", index_col='id')
orig_df = pd.read_csv("/content/secondary_data.csv", sep=";")
test_df = pd.read_csv("/content/test.csv", index_col='id')

In [7]:
train_df = pd.concat([train_df, orig_df], ignore_index=True)

In [8]:
!pip uninstall lightgbm -y

Found existing installation: lightgbm 4.4.0
Uninstalling lightgbm-4.4.0:
  Successfully uninstalled lightgbm-4.4.0


In [9]:
!mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd
# https://www.kaggle.com/code/kirankunapuli/ieee-fraud-lightgbm-with-gpu/notebook

In [10]:
# !pip install lightgbm --install-option=--gpu

In [11]:
# !pip install dask[dataframe]

In [12]:
!pip install 'lightgbm[dask]'

INFO: pip is looking at multiple versions of dask-expr to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.9/241.9 kB 16.9 MB/s eta 0:00:00


In [13]:
predictor = TabularPredictor(label='class',
                            eval_metric='mcc',
                            problem_type='binary').fit(train_df,
                                                       presets='best_quality',
                                                        time_limit=3600*10,
                                                       verbosity=2,
                                                       excluded_model_types=['KNN'],
                                                       ag_args_fit={'num_gpus': 1}
                                                      )
results = predictor.fit_summary()

No path specified. Models will be saved in: "AutogluonModels/ag-20240830_193001"
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.1.1
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Thu Jun 27 21:05:47 UTC 2024
CPU Count:          2
Memory Avail:       10.34 GB / 12.67 GB (81.6%)
Disk Space Avail:   167.55 GB / 201.23 GB (83.3%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will b

(_ray_fit pid=1915) [1000]	valid_set's binary_logloss: 0.0353181	valid_set's mcc: 0.984866
(_ray_fit pid=4796) [1000]	valid_set's binary_logloss: 0.0361311	valid_set's mcc: 0.984644
(_ray_fit pid=7647) [1000]	valid_set's binary_logloss: 0.0356692	valid_set's mcc: 0.984774
(_ray_fit pid=10585) [1000]	valid_set's binary_logloss: 0.0353179	valid_set's mcc: 0.984965
(_ray_fit pid=13539) [1000]	valid_set's binary_logloss: 0.0361859	valid_set's mcc: 0.984374
(_ray_fit pid=16513) [1000]	valid_set's binary_logloss: 0.0365451	valid_set's mcc: 0.984484
(_ray_fit pid=19471) [1000]	valid_set's binary_logloss: 0.03622	valid_set's mcc: 0.984565
(_ray_fit pid=22405) [1000]	valid_set's binary_logloss: 0.0362199	valid_set's mcc: 0.984511


(_dystack pid=1250) 	0.9849	 = Validation score   (mcc)
(_dystack pid=1250) 	5415.15s	 = Training   runtime
(_dystack pid=1250) 	699.84s	 = Validation runtime
(_dystack pid=1250) Fitting model: LightGBM_BAG_L1 ... Training model for up to 448.92s of the 3431.6s of remaining time.
(_dystack pid=1250) 	Memory not enough to fit 8 folds in parallel. Will train 4 folds in parallel instead (Estimated 11.85% memory usage per fold, 47.42%/80.00% total).
(_dystack pid=1250) 	Fitting 8 child models (S1F1 - S1F8) | Fitting with ParallelLocalFoldFittingStrategy (1.0 workers, per: cpus=1, gpus=1, memory=11.85%)
(_dystack pid=1250) 	0.9737	 = Validation score   (mcc)
(_dystack pid=1250) 	415.6s	 = Training   runtime
(_dystack pid=1250) 	27.39s	 = Validation runtime
(_dystack pid=1250) Fitting model: RandomForestGini_BAG_L1 ... Training model for up to 23.57s of the 3006.24s of remaining time.
(_dystack pid=1250) 	Warning: Model is expected to require 1799.8s to train, which exceeds the maximum time 

*** Summary of fit() ***
Estimated performance of each model:
                     model  score_val eval_metric  pred_time_val      fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0      WeightedEnsemble_L3   0.985147         mcc    2125.597421  19276.758692                0.743530          21.534472            3       True          8
1        LightGBMXT_BAG_L1   0.985133         mcc    1543.146187  13493.398151             1543.146187       13493.398151            1       True          1
2      WeightedEnsemble_L2   0.985133         mcc    1543.653075  13501.745367                0.506888           8.347216            2       True          3
3  RandomForestGini_BAG_L2   0.985101         mcc    2124.853891  19255.224220              146.838624        2267.973029            2       True          6
4          LightGBM_BAG_L2   0.984851         mcc    1987.320301  17466.487742                9.305033         479.236551            2       True        

In [14]:
predictor.leaderboard()

,model,score_val,eval_metric,pred_time_val,fit_time,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,0.985147,mcc,2125.597421,19276.758692,0.743530,21.534472,3,True,8
1,LightGBMXT_BAG_L1,0.985133,mcc,1543.146187,13493.398151,1543.146187,13493.398151,1,True,1
2,WeightedEnsemble_L2,0.985133,mcc,1543.653075,13501.745367,0.506888,8.347216,2,True,3
3,RandomForestGini_BAG_L2,0.985101,mcc,2124.853891,19255.224220,146.838624,2267.973029,2,True,6
4,LightGBM_BAG_L2,0.984851,mcc,1987.320301,17466.487742,9.305033,479.236551,2,True,5
5,LightGBM_BAG_L1,0.984431,mcc,434.869081,3493.853040,434.869081,3493.853040,1,True,2
6,LightGBMXT_BAG_L2,0.984268,mcc,2287.575139,20248.439867,309.559871,3261.188676,2,True,4
7,XGBoost_BAG_L2,0.977004,mcc,1995.712477,17386.646365,17.697209,399.395174,2,True,7


In [15]:
y_pred = predictor.predict(test_df)

In [34]:
sub = pd.read_csv('/content/sample_submission.csv')
sub['class'] = y_pred.to_list()
sub.to_csv('submission_secondary_dataset_v2.csv', index=False)

In [21]:
df = pd.read_csv('/content/submission_secondary_dataset.csv')

In [22]:
print(f"열의 개수: {len(df.columns)}")

열의 개수: 2


In [29]:
print(f"행의 개수: {len(df)}")

행의 개수: 2077964
